# DFT Embedding with Qiskit-nature

Nous adoptons le workflow suivant:
1. Effectuer les calculs mean-field (les calculs de référence)
2. Obtenir le Hamiltonien complet du système hors de l'instance `ElectronicStructureProblem`, à partir des intégrales à 1- et 2-corps ($h_{pq}$ et $h_{pqrs}$)
3. Définir un espace actif en utilisant `ActiveSpaceTransformer` et en déduire l'Hamiltonien réduit et l'instance `ElectronicStructureProblem` associé
4. Effectuer le mapping dans l'espace qubit
5. Effectuer le calculs des états excités à partir du solver choisi.

On rappelle que 
\begin{align*}
h_{pq} & = \langle\phi_p|\mathtt{T}_e + \mathtt{U}_{en}|\phi_q\rangle = \int \phi_p(x)^* (\mathtt{T}_e + \mathtt{U}_{en}) \phi_q(x) dx,\\
h_{pqrs} & = \langle\phi_p\phi_q| \mathtt{U}_{ee}|\phi_r\phi_s\rangle = \int \phi_p(x)^* \phi_q(y)^* U_{ee} \phi_r(y) \phi_s(x) dxdy,\\
\phi_j(x)& = \sum_{ij} A_i(x) \text{mo\_coeff}_{i,j},
\end{align*}
avec $\phi_j(x)$ et $A_i(x)$ les orbitales moléculaires et  atomiques respectivement.

De facon plus détaillée on a 

1. **Reference Calculation**

    `driver.run_pyscf()` effectue le calcul Hartree-Fock ou DFT, afin d'obtenir les données nécessaires à la construction de l'Hamiltonien.

2. **Hamiltonian Construction**

    `hamiltonian = ElectronicEnergy(...)` crée un objet `ElectronicEnergy`, représentant l'Hamiltonien électronique.

    `ElectronicIntegrals(...)` est utilisé pour construire les intégrales, cad les termes à 1-électron (+-) et à 2-électrons (++--).

    `S8Integrals(...)` peut-être utilisé pour calculer les intégrales à 2-électrons sous une forme symétrique (format S8).
    
    Cependant, il est plus aisé d'utiliser une instance QCSchema.
    
    ```python
    from qiskit_nature.second_q.formats.qcschema import QCSchema
    from qiskit_nature.second_q.formats import qcschema_translator

    qcschema = driver.to_qcschema(include_dipole=True)
    hamiltonian = qcschema_translator._get_mo_hamiltonian_direct(qcschema)
    ```

3. **Active Space Preparation**

    `active_orbitals` qui spécifie l'ensemble des orbitales qui seront prises en compte dans l'espace actif.

    `ActiveSpaceTransformer` initialise avec le nombre d'orbitales spatiales et d'électrons dans l'espace actif.

    `transformer.prepare_active_space(...)` configure le transformateur en fonction des orbitales actives sélectionnées.

4. **Hamiltonian Reduction**

    `reduced_hamiltonian = transformer.transform_hamiltonian(hamiltonian)` transforme l'Hamiltonien complet du système en un Hamiltonien de l'espace actif, réduisant ainsi la taille du problème.

5. **ElectronicStructureProblem**

    `ElectronicStructureProblem` crée une instance Problem avec `reduced_hamiltonian`.

    Renseigner, pour le Problem, d'autres propriétés pertinentes (comme les énergies orbitales et l'énergie de référence) .

6. **Qubit Mapping and Reduction**

    `ParityMapper` est utilisé pour mapper les opérateurs fermioniques aux opérateurs qubit et on effectue la réduction $\mathbb{Z}_2$.

7. **Excited States Calculation**

    `NumPyEigensolver` est utilisé pour résoudre l'Hamiltonien qubit afin d'obtenir les états excités.

    `ExcitedStatesEigensolver` est configuré pour utiliser `NumPyEigensolver`.

In [1]:
!export OMP_NUM_THREADS=12

In [2]:
# !pip install qiskit-nature[pyscf] -U

In [3]:
# from qiskit_ibm_runtime import QiskitRuntimeService
 
# # Save an IBM Quantum account and set it as your default account.
# QiskitRuntimeService.save_account(channel="ibm_quantum", 
#                                   token="40684e19b34e4d6dc892e172a0bcf3b419484d1c7e88b29e412ed1b72ee2bc7ff01bdf95afd2c6553cf10a69273d7dfa9c3ef948c42246523fbf4bb51fb1351e", 
#                                   set_as_default=True)
 
# # Load saved credentials
# service = QiskitRuntimeService()

## Mean Fields calculations

In [4]:
from qiskit_nature.settings import settings
settings.use_symmetry_reduced_integrals = True

In [20]:
from rdkit import Chem
from rdkit.Chem import AllChem

smiles =  'CC1(C2=CC=CC=C2N(C3=CC=CC=C31)C4=CC=C(C=C4)C5=NC(=NC(=N5)C6=CC=CC=C6)C7=CC=CC=C7)C'
smi_key = 'DMAC-TRZ'
# Defines a molecule from its SMILES string
mol_rdkit = Chem.MolFromSmiles(smiles)

# Add explicit Hs
mol_rdkit = Chem.AddHs(mol_rdkit)

# Generates the initial 3D conformation of the molecule
AllChem.EmbedMolecule(mol_rdkit)

# Optimizes the 3D conformation of the molecule using MMFF - Merck Molecular Force Field
AllChem.MMFFOptimizeMolecule(mol_rdkit, maxIters=200, mmffVariant="MMFF94s")

#Canonicalize the orientation of the conformation
Chem.rdMolTransforms.CanonicalizeMol(mol_rdkit, normalizeCovar=True, ignoreHs=False)

# Convert RDKit molecule to XYZ format
mol_xyz = Chem.MolToXYZBlock(mol_rdkit)

# Remove the first line (number of atoms) from XYZ data
mol_xyz = '\n'.join(mol_xyz.strip().split('\n')[2:])


In [23]:
# setup driver
from qiskit_nature.second_q.drivers import MethodType, PySCFDriver
omega = 1.0
driver = PySCFDriver(
    atom = mol_xyz,
    # atom="O 0.0 0.0 0.115; H 0.0 0.754 -0.459; H 0.0 -0.754 -0.459",
    # atom=open('PSPCz.xyz').read(),
    # atom=open('Thiophene.xyz').read(),
    # atom=open('BODIPY.xyz').read(),
    basis="6-31g*",
    method=MethodType.RKS,
    xc_functional=f"ldaerf + lr_hf({omega})",
    xcf_library="xcfun",
)

In [24]:
'''
1. Run the reference calculation
'''
driver.run_pyscf()
mf = driver._calc

In [26]:
print(f'Le nombre total d\'électrons est {driver._mol.nelectron} \
et le nombre total d\'électrons (alpha, beta) est {driver._mol.nelec}\n')
print(f'Les index (0-Based) du (HOMO,LUMO) sont {driver._mol.nelectron//2 -1, driver._mol.nelectron//2}\n')
print(f'Le nombre d\'orbitales atomiques, dans la base {driver._mol.basis}, est {driver._mol.nao}\n')
print(f'L\'énergie nucléaire vaut {driver._mol.energy_nuc()} Ha')

Le nombre total d'électrons est 272 et le nombre total d'électrons (alpha, beta) est (136, 136)

Les index (0-Based) du (HOMO,LUMO) sont (135, 136)

Le nombre d'orbitales atomiques, dans la base 6-31g*, est 616

L'énergie nucléaire vaut 3918.305308283061 Ha


In [27]:
import plotly.express as px

# Plot the MO Occupations
fig = px.line(y=mf.mo_occ, markers=True, title="Molecular Orbital (MO) Occupations")
fig.update_layout(xaxis_title="Orbital Index (0-Based)", yaxis_title="MO Occupation")
fig.show()

In [28]:
from pyscf.data.nist import HARTREE2EV as au2ev 

lumo_idx = mf.mo_occ.tolist().index(0.)
homo_idx = lumo_idx - 1
print(f'Energie du Homo = {mf.mo_energy[homo_idx] * au2ev} eV')
print(f'Energie de Lumo = {mf.mo_energy[lumo_idx] * au2ev} eV')
print(f'Energie du gap Homo-Lumo = {abs(mf.mo_energy[homo_idx] - mf.mo_energy[lumo_idx]) * au2ev} eV')

Energie du Homo = -7.652638241843805 eV
Energie de Lumo = 1.0054711133428857 eV
Energie du gap Homo-Lumo = 8.65810935518669 eV


In [29]:
# Plot the MO Energies (i.e. eigenvalues of the Fock matrix)
fig = px.line(y=mf.mo_energy, markers=True, title="Molecular Orbital (MO) Energies (a.u.)")
fig.update_layout(xaxis_title="Orbital Index (0-Based)", yaxis_title="MO Energies (a.u.)")
fig.show()

In [30]:
mf.dump_scf_summary()

**** SCF Summaries ****
Total Energy =                       -1593.784007953246146
Nuclear Repulsion Energy =            3918.305308283061095
One-electron Energy =                -9970.885586827305815
Two-electron Coulomb Energy =         4678.642672013977062
DFT Exchange-Correlation Energy =     -219.846401422978829


In [ ]:
from pyscf.tools import molden

with open('mytest.molden', 'w') as f1:
    molden.header(driver, f1)
    molden.orbital_coeff(driver, f1, mf.mo_coeff, ene=mf.mo_energy, occ=mf.mo_occ)

## ElectronicEnergy calculation

La classe `ElectronicEnergy` implémente le Hamiltonien 

$$ H_{el} = \sum_{p, q} h_{pq} a^\dagger_p a_q
         + \sum_{p, q, r, s} g_{pqrs} a^\dagger_p a^\dagger_q a_r a_s ,$$

où $h_{pq}$ et $g_{pqrs}$ sont les intégrales électroniques à un et deux corps,
stocké dans le conteneur `qiskit_nature.second_q.operators.ElectronicIntegrals`.
Lorsqu'il s'agit de coefficients séparés pour les électrons de spin $\alpha$- et $\beta$,
l'Hamiltonien à spin sans restriction (UHF) peut être obtenu à partir de celui ci-dessus de manière simple.

On peut construire une instance de cet Hamiltonien de plusieurs manières.

1. Avec une instance existante de `qiskit_nature.second_q.operators.ElectronicIntegrals` :

```python
   intégrales : ElectronicIntegrals = ...

   hamiltonien = ElectronicEnergy (intégrales, constantes = {"nuclear_repulsion_energy": 1.0})
```
2. À partir d’un ensemble brut de matrices de coefficients intégraux 
    * h1_a: the alpha-spin one-body coefficients.
    * h2_aa: the alpha-alpha-spin two-body coefficients.
    * h1_b: the beta-spin one-body coefficients.
    * h2_bb: the beta-beta-spin two-body coefficients.
    * h2_ba: the beta-alpha-spin two-body coefficients.:

```python
    # en supposant que les intégrales à un et deux corps ont été préalablement calculées
    h1_a, h2_aa, h1_b, h2_bb, h2_ba = ...

    hamiltonien = ElectronicEnergy.from_raw_integrals(h1_a, h2_aa, h1_b, h2_bb, h2_ba)
    hamiltonian.nuclear_repulsion_energy = 1.0
```
3. À partir d'un instance QSChema
```python
    from qiskit_nature.second_q.formats.qcschema import QCSchema
    from qiskit_nature.second_q.formats import qcschema_translator 

    qcschema = driver.to_qcschema(include_dipole=True)
    hamiltonian = qcschema_translator._get_mo_hamiltonian_direct(qcschema)

```

Il est à noter que nous avons spécifié l'énergie de répulsion nucléaire comme un décalage d'énergie constant dans ce qui précède. Ce terme ne sera pas inclus dans l'opérateur qubit mappé puisqu'il s'agit d'un terme de décalage constant et qu'il ne nécessite pas d'erreurs lors de la mesure sur un appareil quantique. Il est cependant possible d'inclure des termes d'énergie constante à l'intérieur du conteneur `qiskit_nature.second_q.operators.ElectronicIntegrals`, si l'on souhait qu'il soit inclus dans l'opérateur qubit, lors du mapping de l'opérateur de 2e quantification à l'espace qubit.
 ``` python
    from qiskit_nature.second_q.operators import PolynomialTensor

        e_nuc = hamiltonian.nuclear_repulsion_energy
        hamiltonian.electronic_integrals.alpha += PolynomialTensor({"": e_nuc})
        hamiltonian.nuclear_repulsion_energy = None
```

In [31]:
# def compute_integrals(driver):
#     """Compute 1-electron and 2-electron integrals
#     The return is formatted as
#     [numpy.ndarray]*2 numpy array h_{pq} for alpha and beta blocks
#     [numpy.ndarray]*3 numpy array storing h_{pqrs} for alpha-alpha, alpha-beta, beta-beta blocks

#     Args:
#         driver : The PySCF object to calculated UHF integrals for.

#     Returns:
#         List[array], List[array]: One and two body integrals
#     """
#     from pyscf import ao2mo
#     import numpy as np

    # step 1 : find nao, nmo (atomic orbitals & molecular orbitals)

    # molecular orbitals (alpha and beta will be the same)
    # Lets take alpha blocks to find the shape and things

    # # molecular orbitals
    # mf = driver._calc
    # mo_coeff, mo_coeff_b = driver._expand_mo_object(mf.mo_coeff, array_dimension=3)
    # nmo = mo_coeff[0].shape[1]
    # # # atomic orbitals
    # # nao = mo_coeff[0].shape[0]
    # nao = len(mo_coeff)
    # nmo = nao

    # # step 2 : obtain Hcore Hamiltonian (equivalent to int1e_kin + int1e_nuc) in ao basis
    # hcore = mf.get_hcore() 

    # # step 3 : obtain two-electron integral in ao basis
    # eri = ao2mo.restore(8, mf._eri, nao)

    # # # step 4 : create the placeholder for the matrices
    # # # one-electron matrix (alpha, beta)
    # # hpq = []

    # # step 5 : do the mo transformation
    # # step the mo coeff alpha and beta
    # mo_a = mo_coeff
    # mo_b = mo_coeff_b

    # mo transform the hcore
    # hpq.append(mo_a.T.dot(hcore).dot(mo_a))
    # if mo_coeff_b is not None:
        # hpq.append(mo_b.T.dot(hcore).dot(mo_b))
    # h1_a = np.dot(np.dot(mo_coeff.T, hcore), mo_coeff)
    # if mo_coeff_b is not None:
    #     h1_b = np.dot(np.dot(mo_coeff_b.T, hcore), mo_coeff_b)
    # else:
    #     h1_b = None

    # # mo transform the two-electron integrals
    # eri_a = ao2mo.incore.full(eri, mo_a)
    # eri_b = ao2mo.incore.full(eri, mo_b)
    # eri_ba = ao2mo.incore.general(eri, (mo_a, mo_a, mo_b, mo_b), compact=False)

    # # Change the format of integrals (full)
    # eri_a = ao2mo.restore(1, eri_a, nmo)
    # eri_b = ao2mo.restore(1, eri_b, nmo)
    # eri_ba = eri_ba.reshape(nmo, nmo, nmo, nmo)

    # h2_aa = driver._mol.intor("int2e", aosym=8)
    # h2_aa = ao2mo.restore(8, mf._eri, len(mo_coeff))
    # h2_aa = ao2mo.incore.full(eri, mo_a)
    # if mo_coeff_b is not None:
    #     h2_bb = ao2mo.incore.full(eri, mo_a)
    #     h2_ba = ao2mo.incore.general(eri, (mo_b, mo_b, mo_a, mo_a), compact=False)
    # else:
    #     h2_bb = None
    #     h2_ba = None

    # # convert this into the physicist ordering for OpenFermion
    # two_body_integrals_a = np.asarray(eri_a.transpose(0, 2, 3, 1), order='C')
    # two_body_integrals_b = np.asarray(eri_b.transpose(0, 2, 3, 1), order='C')
    # two_body_integrals_ab = np.asarray(eri_ba.transpose(0, 2, 3, 1), order='C')

    # # Gpqrs has alpha, alphaBeta, Beta blocks
    # Gpqrs = (two_body_integrals_a, two_body_integrals_ab, two_body_integrals_b)

    # return h1_a, h2_aa, h1_b, h2_bb, h2_ba


In [32]:
'''
 2. Build the total Hamiltonian outside of a Problem instance
'''
# import numpy as np
# from pyscf import ao2mo
# from qiskit_nature.second_q.operators.symmetric_two_body import fold
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
# from qiskit_nature.second_q.operators import ElectronicIntegrals, PolynomialTensor

# mo_coeff, mo_coeff_b = driver._expand_mo_object(mf.mo_coeff, array_dimension=3)
# h1_a = mf.get_hcore()
# h1_a = np.dot(np.dot(mo_coeff.T, h1_a), mo_coeff)
# if mo_coeff_b is not None:
#     h1_b = np.dot(np.dot(mo_coeff_b.T, h1_a), mo_coeff_b)
# else:
#     h1_b = None

# # h2_aa = driver._mol.intor("int2e", aosym=8)
# h2_aa = ao2mo.restore(8, mf._eri, len(mo_coeff))
# h2_aa = fold(ao2mo.full(driver._mol, mo_coeff, aosym=4))
# if mo_coeff_b is not None:
#     h2_bb = fold(ao2mo.full(driver._mol, mo_coeff_b, aosym=4))
#     h2_ba = fold(ao2mo.general(
#             driver._mol,
#             [mo_coeff_b, mo_coeff_b, mo_coeff, mo_coeff],
#             aosym=4,
#         )
#     )
# else:
#     h2_bb = None
#     h2_ba = None

# h1_a, h2_aa, h1_b, h2_bb, h2_ba = compute_integrals(driver)
    
# hamiltonian = ElectronicEnergy.from_raw_integrals(
#     h1_a,
#     h2_aa,
#     h1_b,
#     h2_bb,
#     h2_ba
# )

# hamiltonian = ElectronicEnergy.from_raw_integrals(compute_integrals(driver))
# # To included the constant nuclear_repulsion_energy in the second_q_op
# e_nuc = hamiltonian.nuclear_repulsion_energy
# hamiltonian.electronic_integrals.alpha += PolynomialTensor({"": e_nuc})
# hamiltonian.nuclear_repulsion_energy = driver._mol.energy_nuc()

In [33]:
'''
 2. Build the total Hamiltonian outside of a Problem instance
 Use of QCSchema
'''
from qiskit_nature.second_q.formats.qcschema import QCSchema
from qiskit_nature.second_q.formats import qcschema_translator 
from qiskit_nature.second_q.operators import PolynomialTensor

qcschema = driver.to_qcschema(include_dipole=True)
hamiltonian = qcschema_translator._get_mo_hamiltonian_direct(qcschema)

hamiltonian.nuclear_repulsion_energy = driver._mol.energy_nuc()

# # To included the constant nuclear_repulsion_energy in the second_q_op
# e_nuc = hamiltonian.nuclear_repulsion_energy
# hamiltonian.electronic_integrals.alpha += PolynomialTensor({"": e_nuc})
# # hamiltonian.nuclear_repulsion_energy = None

MemoryError: Unable to allocate 135. GiB for an array with shape (18056935666,) and data type float64

## Active-Space reduction

L'énergie électronique totale est définie par, 
$$
    E = \sum_{pq} h_{pq}D_{pq} + \frac12\sum_{pqrs} g_{pqrs}d_{pqrs},
$$
où $h_{pq}$ et $g_{pqrs}$ sont les intégrales à un et deux électrons, respectivement, et $D$ et $d$ sont les opérateurs statistiques (ou matrices de densité) à une et deux particules.

Pour définir un espace actif, nous divisons l'opérateur à un électron en une partie active et une partie inactive, $D=D^A+D^I$. Dans la base des MOs, cette dernière se simplifie à $D^I_{iq}=2\delta_{iq}$, où nous utilisons la notation des indices de Helgaker dans laquelle $i,j,k,l$ désignent *inactif*, $u,v,x,y$ désignent *actif* et $p,q,r,s$ désignent les MOs *général*. On obtient ainsi,
$$
    E = E^I + \sum_{uv} F^I_{uv} D_{uv}^A + \frac12 \sum_{uvxy} g_{uvxy}d_{uvxy}^A,
$$
où l'**opérateur de Fock inactif** est définie comme
$$ F^I_{pq} = h_{pq} + \sum_i (2 g_{iipq} - g_{iqpi}),$$
et l'**énergie inactive** est donnée par
$$ E^I = \sum_j h_{ji} + F^I_{jj} = \frac12 \sum_{ij} \Big(h_{ij} + F^I_{jj}\Big) D^I_{ij} .$$

C'est ce qui est implémenté dans `qiskit_nature.second_q.transformers.ActiveSpaceTransformer`.

On note que 
* l'opérateur de Fock *inactive* L'opérateur de Fock, $F^I$, remplace les intégrales à un électron, $h_{pq}$, 
* les opérateurs statistiques à un et deux électrons, $D^A$ et $d^A$, remplacent $D$ et $d$, et le décalage d'énergie constante, $E^I$, est ajouté.

Par conséquent, l'Hamiltonien simulé dans le calculateur quantique  la forme suivante
$$
    \mathtt{H}_{qc} = \sum_{uv} F^I_{uv} a^\dagger_u a_v + \sum_{uvxy} g_{uvxy} 
    a^\dagger_u a^\dagger_v a_x a_y .
$$
En utilisant l'opérateur de Fock inactif à la place des intégrales à un électron, la description de l'espace actif contient un potentiel effectif généré par les électrons inactifs. Par conséquent, cette méthode permet l'exclusion des électrons non centraux tout en conservant une description de haute qualité du système.

On rappelle que l'opérateur de Fock est défini comme
$$ F_{pq} = h_{pq} + J_{pq} - K_{pq},$$
avec $J_{qr} = \sum g_{pqrs} D_{ps}$ et $K_{pr} = \sum g_{pqrs} D_{qs}$ est opérateurs de Coulomb et d'échange respectivement.

In [ ]:
''' 
3. Prepare the active space by initializing it with the total problem size information
'''
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
import numpy as np

# Somes variables needed
total_num_particles = driver._mol.nelec
total_num_spatial_orbitals = driver._mol.nao # hamiltonian.register_length
total_num_electrons = driver._mol.nelectron

# Alpha-spin
num_alpha = total_num_particles[0] # qcschema.properties.calcinfo_nalpha
orbital_occupations = np.asarray([1.0] * num_alpha 
                                 + [0.0] * (total_num_spatial_orbitals - num_alpha))
# Beta-spin
num_beta = total_num_particles[1] # , qcschema.properties.calcinfo_nbeta
orbital_occupations_b = np.asarray([1.0] * num_beta 
                                   + [0.0] * (total_num_spatial_orbitals - num_beta))
# Initialize the transformer
transformer =  ActiveSpaceTransformer(2,2)

# Prepare the active space
transformer.prepare_active_space(total_num_particles, total_num_spatial_orbitals,
                                 occupation_alpha=orbital_occupations, 
                                 occupation_beta=orbital_occupations_b,)
# Determine the active space
as_orbitals = transformer._determine_active_space(total_num_electrons, total_num_spatial_orbitals)[0]

# Use the active space transformer to reduce the total Hamiltonian to the active space
reduced_hamiltonian = transformer.transform_hamiltonian(hamiltonian)

In [ ]:
print(as_orbitals)
print(reduced_hamiltonian.second_q_op())

[21, 22]
Fermionic Operator
number spin orbitals=4, number terms=72
  0.17376282576625582 * ( +_0 +_0 -_0 -_0 )
+ 1.3819719814389634e-06 * ( +_0 +_0 -_1 -_0 )
+ 1.3819719814389634e-06 * ( +_0 +_1 -_0 -_0 )
+ 0.1555131839170142 * ( +_0 +_1 -_1 -_0 )
+ 0.17376282576625582 * ( +_0 +_2 -_2 -_0 )
+ 1.3819719814389634e-06 * ( +_0 +_2 -_3 -_0 )
+ 1.3819719814389634e-06 * ( +_0 +_3 -_2 -_0 )
+ 0.1555131839170142 * ( +_0 +_3 -_3 -_0 )
+ 1.3819719814389634e-06 * ( +_0 +_0 -_0 -_1 )
+ 0.03465890227078017 * ( +_0 +_0 -_1 -_1 )
+ 0.03465890227078017 * ( +_0 +_1 -_0 -_1 )
+ 4.7874329104323004e-08 * ( +_0 +_1 -_1 -_1 )
+ 1.3819719814389634e-06 * ( +_0 +_2 -_2 -_1 )
+ 0.03465890227078017 * ( +_0 +_2 -_3 -_1 )
+ 0.03465890227078017 * ( +_0 +_3 -_2 -_1 )
+ 4.7874329104323004e-08 * ( +_0 +_3 -_3 -_1 )
+ 1.3819719814389634e-06 * ( +_1 +_0 -_0 -_0 )
+ 0.03465890227078017 * ( +_1 +_0 -_1 -_0 )
+ 0.03465890227078017 * ( +_1 +_1 -_0 -_0 )
+ 4.7874329104323004e-08 * ( +_1 +_1 -_1 -_0 )
+ 1.3819719814389634e-06

In [ ]:
from pyscf.tools import cubegen

# Output cube files for active orbitals that can read by Jmol. Avogadro, Gabedit
for i in as_orbitals:
    cubegen.orbital(
        driver._mol, 
        f'Test_DFT_{i+1}.cube', 
        mf.mo_coeff[:, i])

In [ ]:
''' 
4. Setup the Problem instance
'''
from qiskit_nature.second_q.problems import ElectronicBasis, ElectronicStructureProblem
from typing import cast

reduced_hamiltonian.constants["inactive energy"] = mf.e_tot - driver._mol.energy_nuc()

problem = ElectronicStructureProblem(reduced_hamiltonian)
problem.basis = ElectronicBasis.MO
problem.reference_energy = mf.e_tot
problem.num_spatial_orbitals = transformer._num_spatial_orbitals

problem.orbital_occupations = np.diag(
    cast(np.ndarray, transformer._active_density.alpha["+-"])
)[transformer._active_alpha_indices]
problem.orbital_occupations_b = np.diag(
    cast(np.ndarray, transformer._active_density.beta["+-"])
)[transformer._active_beta_indices]
problem.num_particles = (
    round(sum(problem.orbital_occupations)),
    round(sum(problem.orbital_occupations_b)),
)

mo_energy, mo_energy_b = driver._expand_mo_object(driver._calc.mo_energy)
if mo_energy is not None:
    problem.orbital_energies = mo_energy[transformer._active_alpha_indices]
if mo_energy_b is not None:
    problem.orbital_energies_b = mo_energy_b[transformer._active_beta_indices]

In [ ]:
''' 
5. Setup Problem Electronic properties
'''
from qiskit_nature.second_q.properties import (
    AngularMomentum,
    ElectronicDensity,
    Magnetization,
    ParticleNumber,
    ElectronicDipoleMoment
)
from qiskit_nature.second_q.operators import Tensor

for prop in problem.properties:
    if isinstance(prop, (Magnetization, ParticleNumber)):
        problem.properties.add(prop.__class__(problem.num_spatial_orbitals))
    
    elif isinstance(prop, ElectronicDipoleMoment):
        problem.properties.electronic_dipole_moment = (
            _transform_electronic_dipole_moment(
                prop,
                transformer._density_total,
                transformer._active_density,
                transformer._active_basis,
                transformer.__class__.__name__,
            )
        )

    elif isinstance(prop, ElectronicDensity):
        transformed = transformer._active_basis.transform_electronic_integrals(prop)
        problem.properties.electronic_density = ElectronicDensity(
            transformed.alpha, transformed.beta, transformed.beta_alpha
        )

    elif isinstance(prop, AngularMomentum):
        if prop.overlap is None:
            # only the size needs to be changed
            problem.properties.add(prop.__class__(problem.num_spatial_orbitals))
            continue

        if isinstance(transformer._active_basis.coefficients, ElectronicIntegrals):
            coeff_alpha = transformer._active_basis.coefficients.alpha["+-"]
            coeff_beta: Tensor
            if transformer._active_basis.coefficients.beta.is_empty():
                coeff_beta = coeff_alpha
            else:
                coeff_beta = transformer._active_basis.coefficients.beta["+-"]

            problem.properties.angular_momentum = AngularMomentum(
                problem.num_spatial_orbitals,
                coeff_alpha.transpose() @ prop.overlap @ coeff_beta,
            )

norb = problem.num_spatial_orbitals
problem.properties.particle_number = ParticleNumber(norb)
problem.properties.magnetization = Magnetization(norb)
problem.properties.angular_momentum = AngularMomentum(norb)
problem.properties.electronic_density = ElectronicDensity.from_orbital_occupation(
    problem.orbital_occupations, 
    problem.orbital_occupations_b
)


In [ ]:
# from qiskit_nature.second_q.operators import ElectronicIntegrals

# # Total electronic density
# total_density = ElectronicDensity.from_orbital_occupation(
#                 orbital_occupations,
#                 orbital_occupations_b,
#                 include_rdm2=False,
#             )

# density_total = ElectronicIntegrals.from_raw_integrals(
#     np.diag(orbital_occupations), h1_b=np.diag(orbital_occupations_b))
# density_actitve = transformer.active_density
# active_basis = transformer.active_basis


In [ ]:
''' 
6. mapping the Hamiltonian to spin space
'''
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper, TaperedQubitMapper

mapper = ParityMapper(num_particles=problem.num_particles)
mapper = problem.get_tapered_mapper(mapper)

Red_hamil_z2qubit = mapper.map(reduced_hamiltonian.second_q_op())
print(f"Number of items in the PM Z2 Pauli list:", len(Red_hamil_z2qubit))
Red_hamil_z2qubit


Number of items in the PM Z2 Pauli list: 9


SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ', 'XI', 'XZ', 'IX', 'ZX', 'XX'],
              coeffs=[-6.43647118e-01+0.j,  6.86603199e-02+0.j, -6.86603199e-02+0.j,
 -1.13712725e-02+0.j,  1.46372152e-06+0.j,  6.00215294e-07+0.j,
  1.46372152e-06+0.j, -6.00215294e-07+0.j,  4.58829177e-02+0.j])

In [ ]:
# ''' 
# 8. Setup solver 
# '''
# from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
# from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# solver = NumPyMinimumEigensolver(filter_criterion=problem.get_default_filter_criterion())
# # solver = NumPyMinimumEigensolver()
# # solver.filter_criterion = lambda state, val, aux: np.isclose(
# #     aux["ParticleNumber"][0], 4.0
# #     )

# algo = GroundStateEigensolver(mapper, solver)


In [ ]:
# result = algo.solve(problem)

# print(f"Total ground state energy = {result.total_energies[0]:.4f}")

In [ ]:
''' 
7. Setup solver 
'''

from qiskit_algorithms import NumPyEigensolver
from qiskit_nature.second_q.algorithms import ExcitedStatesEigensolver

# Setup the solver
numpy_solver = NumPyEigensolver(k=6, filter_criterion=problem.get_default_filter_criterion())

# solver = NumPyEigensolver(k=10)
# solver.filter_criterion = lambda state, val, aux: np.isclose(
#     aux["ParticleNumber"][0], 4.0
# )

numpy_ES_solver = ExcitedStatesEigensolver(mapper, numpy_solver)
NP_ES = numpy_ES_solver.solve(problem)

print(NP_ES)


=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -3070.943752110647
  - computed part:      -0.77705913217
  - ActiveSpaceTransformer extracted energy part: -1535.327058640119
  - inactive energy extracted energy part: -1534.839634338359
~ Nuclear repulsion energy (Hartree): 858.64065153521
> Total ground state energy (Hartree): -2212.303100575437
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -3070.77582845083
> Total excited state energy (Hartree): -2212.135176915619
  2: 
* Electronic excited state energy (Hartree): -3070.654185536302
> Total excited state energy (Hartree): -2212.013534001092
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
  1:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
  2:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
